In [1]:
# Filter in the api call, 2018: 9/11/18-10/11/18

import pandas as pd
import numpy as np
import datetime
import pandas as pd
from googleads import adwords
import os
from apiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials

week="_September_SP" 

Start_Date_overall='2018-09-11'
End_Data_overall='2018-10-11'

Start_Date_1='2018-09-11'
End_Date_1='2018-09-25'

Start_Date_2='2018-09-26'
End_Date_2='2018-10-11'


today_str=str(datetime.datetime.now().date())
writer_folder="/home/jian/Projects/Saatva/To_Lexie_and_Connor/output/running_on_"+today_str+"/"
try:
    os.stat(writer_folder)
except:
    os.mkdir(writer_folder)

writer_folder="/home/jian/Projects/Saatva/To_Lexie_and_Connor/output/running_on_"+today_str+"/"+week[1:]+"/"
try:
    os.stat(writer_folder)
except:
    os.mkdir(writer_folder)

In [2]:
def initialize_analyticsreporting(KEY_FILE_LOCATION):
    """Initializes an Analytics Reporting API V4 service object.
    Args:
    KEY_FILE_LOCATION: str, ServiceAccount key file
    Returns:
    An authorized Analytics Reporting API V4 service object.
    """

    SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']

    credentials = ServiceAccountCredentials.from_json_keyfile_name(
        KEY_FILE_LOCATION, scopes=SCOPES)

    # Build the service object.
    analytics = build('analytics', 'v4', credentials=credentials)

    return analytics


def get_report_Google_Search(analytics, VIEW_ID, start_date, end_date, ga_metrics, ga_dimensions, next_page='0'):
    """Queries the Analytics Reporting API V4.

    Args:
    analytics: An authorized Analytics Reporting API V4 service object.
    VIEW_ID: str
    start_date: str
    end_date: str
    ga_metrics: list of str
    ga_dimensions: list of str
    Returns:
    The Analytics Reporting API V4 response.
    """
    return analytics.reports().batchGet(
        body = {
            'reportRequests': [
                {
                    'viewId': VIEW_ID,
                    'pageSize': 10000,
                    'pageToken': next_page,
                    'dateRanges': [{'startDate': start_date, 
                                    'endDate': end_date}],
                    'metrics': [{'expression': m} for m in ga_metrics],
                    'dimensions': [{'name': d} for d in ga_dimensions],
                    "dimensionFilterClauses":[{"operator": "AND",
                                               "filters": [
                                                   {
                                                   "dimensionName": "ga:medium",
                                                   "operator": "IN_LIST",
                                                   "expressions": ['(none)','cpc','organic','(not set)'],
                                                   "caseSensitive":False},
                                                   
                                                   {"dimensionName": "ga:source",
                                                   "operator": "PARTIAL",
                                                   "expressions": "oogle",
                                                   "caseSensitive":False}]
                                              }],
                }]
        }
    ).execute()

def get_report(analytics, VIEW_ID, start_date, end_date, ga_metrics, ga_dimensions, next_page='0'):
    """Queries the Analytics Reporting API V4.

    Args:
    analytics: An authorized Analytics Reporting API V4 service object.
    VIEW_ID: str
    start_date: str
    end_date: str
    ga_metrics: list of str
    ga_dimensions: list of str
    Returns:
    The Analytics Reporting API V4 response.
    """
    return analytics.reports().batchGet(
        body = {
            'reportRequests': [
                {
                    'viewId': VIEW_ID,
                    'pageSize': 10000,
                    'pageToken': next_page,
                    'dateRanges': [{'startDate': start_date, 
                                    'endDate': end_date}],
                    'metrics': [{'expression': m} for m in ga_metrics],
                    'dimensions': [{'name': d} for d in ga_dimensions]
                }]
        }
    ).execute()

def response_to_dataframe(response):
    """Parses and prints the Analytics Reporting API V4 response.

    Args:
    response: An Analytics Reporting API V4 response.
    
    Returns:
    An pandas DataFrame
    """
    report = response['reports'][0]
    df_data = []
    columnHeader = report.get('columnHeader', {})
    dimensionHeaders = columnHeader.get('dimensions', [])
    metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])

    for row in report.get('data', {}).get('rows', []):
        row_data = dict()
        dimensions = row.get('dimensions', [])
        dateRangeValues = row.get('metrics', [])

        for header, dimension in zip(dimensionHeaders, dimensions):
            row_data[header] = dimension

        for i, values in enumerate(dateRangeValues):
            #print 'Date range: ' + str(i)
            for metricHeader, value in zip(metricHeaders, values.get('values')):
                row_data[metricHeader.get('name')] = value
        df_data.append(row_data)
        
    return pd.DataFrame(df_data)

def get_report_over_10000_rows_Google_Search(analytics, VIEW_ID, start_date, end_date, 
                               ga_metrics, ga_dimensions):
    """Queries the Analytics Reporting API V4 for reports containing over 10000 
    rows.

    Args:
    analytics: An authorized Analytics Reporting API V4 service object.
    VIEW_ID: str
    start_date: str
    end_date: str
    ga_metrics: list of str
    ga_dimensions: list of str
    Returns:
    The Analytics Reporting API V4 response.
    """
    response = get_report_Google_Search(analytics, VIEW_ID, start_date, end_date, ga_metrics, ga_dimensions)
    products_df = response_to_dataframe(response)
    while 'nextPageToken' in response['reports'][0]:
        next_page_token = response['reports'][0]['nextPageToken']
        response = get_report_Google_Search(analytics, VIEW_ID, start_date, end_date, 
                              ga_metrics, ga_dimensions, next_page=next_page_token)
        df = response_to_dataframe(response)
        products_df = products_df.append(df)
    return products_df

def get_report_over_10000_rows(analytics, VIEW_ID, start_date, end_date, 
                               ga_metrics, ga_dimensions):
    """Queries the Analytics Reporting API V4 for reports containing over 10000 
    rows.

    Args:
    analytics: An authorized Analytics Reporting API V4 service object.
    VIEW_ID: str
    start_date: str
    end_date: str
    ga_metrics: list of str
    ga_dimensions: list of str
    Returns:
    The Analytics Reporting API V4 response.
    """
    response = get_report(analytics, VIEW_ID, start_date, end_date, ga_metrics, ga_dimensions)
    products_df = response_to_dataframe(response)
    while 'nextPageToken' in response['reports'][0]:
        next_page_token = response['reports'][0]['nextPageToken']
        response = get_report(analytics, VIEW_ID, start_date, end_date, 
                              ga_metrics, ga_dimensions, next_page=next_page_token)
        df = response_to_dataframe(response)
        products_df = products_df.append(df)
    return products_df

# Google Analytics

In [3]:
analytics = initialize_analyticsreporting('/home/jian/Projects/Saatva/GA/GA_api_jubapluscc@gmail.com.json')

def num_func_trans(df):
    if 'ga:sessions' in df.columns:
        df['ga:sessions']=df['ga:sessions'].apply(lambda x: int(x.replace(",","")))
        df['ga:sessions']=df['ga:sessions'].astype(int)

    if 'ga:adClicks' in df.columns:
        df['ga:adClicks']=df['ga:adClicks'].apply(lambda x: int(x.replace(",","")))
        df['ga:adClicks']=df['ga:adClicks'].astype(int)

    if 'ga:impressions' in df.columns:
        df['ga:impressions']=df['ga:impressions'].apply(lambda x: int(x.replace(",","")))
        df['ga:impressions']=df['ga:impressions'].astype(int)
    
    if 'ga:transactions' in df.columns:
        df['ga:transactions']=df['ga:transactions'].apply(lambda x: int(x.replace(",","")))
        df['ga:transactions']=df['ga:transactions'].astype(int)
    
    if 'ga:transactionRevenue' in df.columns:
        df['ga:transactionRevenue']=df['ga:transactionRevenue'].apply(lambda x: float(x.replace(",","")))
        df['ga:transactionRevenue']=df['ga:transactionRevenue'].astype(float)

    
    return df


In [4]:

ga_data_1=get_report_over_10000_rows_Google_Search(analytics=analytics, VIEW_ID='122377905',start_date=Start_Date_1, end_date=End_Date_1,
                                            ga_metrics=['ga:sessions','ga:adClicks','ga:impressions','ga:transactions','ga:transactionRevenue'],
                                            ga_dimensions=['ga:dateHour','ga:source','ga:medium'])

ga_data_2=get_report_over_10000_rows_Google_Search(analytics=analytics, VIEW_ID='122377905',start_date=Start_Date_2, end_date=End_Date_2,
                                            ga_metrics=['ga:sessions','ga:adClicks','ga:impressions','ga:transactions','ga:transactionRevenue'],
                                            ga_dimensions=['ga:dateHour','ga:source','ga:medium'])


ga_data_saatva_GoogleSearch=ga_data_1.append(ga_data_2)
ga_data_saatva_GoogleSearch['View']="Saatva.com"
ga_data_saatva_GoogleSearch['View_ID']="122377905"
ga_data_saatva_GoogleSearch.shape

(1183, 10)

In [5]:
# Saatvamattress.com
# dateHour, sourceMedium      sessions, adClicks, impressions
ga_data_1=get_report_over_10000_rows_Google_Search(analytics=analytics, VIEW_ID='33605697',start_date=Start_Date_1, end_date=End_Date_1,
                                            ga_metrics=['ga:sessions','ga:adClicks','ga:impressions','ga:transactions','ga:transactionRevenue'],
                                            ga_dimensions=['ga:dateHour','ga:source','ga:medium'])

ga_data_2=get_report_over_10000_rows_Google_Search(analytics=analytics, VIEW_ID='33605697',start_date=Start_Date_2, end_date=End_Date_2,
                                            ga_metrics=['ga:sessions','ga:adClicks','ga:impressions','ga:transactions','ga:transactionRevenue'],
                                            ga_dimensions=['ga:dateHour','ga:source','ga:medium'])


ga_data_SM_GoogleSearch=ga_data_1.append(ga_data_2)
ga_data_SM_GoogleSearch['View']="Saatvamattress.com"
ga_data_SM_GoogleSearch['View_ID']="33605697"
ga_data_SM_GoogleSearch.shape

(1498, 10)

In [6]:
ga_data=ga_data_SM_GoogleSearch.append(ga_data_saatva_GoogleSearch)
ga_data=num_func_trans(ga_data)

In [7]:
ga_data.head(2)

,ga:adClicks,ga:dateHour,ga:impressions,ga:medium,ga:sessions,ga:source,ga:transactionRevenue,ga:transactions,View,View_ID
0,0,2018091100,0,cpc,471,google,4352.0,3,Saatvamattress.com,33605697
1,0,2018091100,0,organic,46,google,5304.0,4,Saatvamattress.com,33605697


# Google AdWords

In [41]:
def get_client(config_path='googleads_saatva.yaml'):
    return adwords.AdWordsClient.LoadFromStorage(config_path)

def download_report_as_stream(client, report_type, fields, start_date, end_date):
    '''Download Adwords report, then save as csv file to base_dir
    Args:
        client: adwords.AdWordsClient.LoadFromStorage()
        report_type: str eg. 'KEYWORDS_PERFORMANCE_REPORT'
        fields: list eg. ["CampaignId", "CampaignName", ... ]
        start_date: str, eg. '2018-02-11'
        end_date: str, eg.
    Return:
        str, path to csv file
    '''
    report_downloader = client.GetReportDownloader(version='v201802')
    # Create report definition.
    report = {
        'reportName': report_type,
        'dateRangeType': 'CUSTOM_DATE',
        'reportType': report_type,
        'downloadFormat': 'CSV',
        'selector': {
            'fields': fields,
            'dateRange': {
                'min': start_date,
                'max': end_date},
            'predicates': [{'field': 'AdvertisingChannelType',
                            'operator': 'EQUALS',
                            'values': ["Search"]}],
        }
    }

    # save output
    return report_downloader.DownloadReportAsStream(report, 
        skip_report_header=True, skip_column_header=False,
        skip_report_summary=True)



In [42]:
client_Saatva = get_client(config_path='/home/jian/Projects/Saatva/To_Lexie_and_Connor/googleads_saatva_JL_Saatva.yaml')
report_type = 'CAMPAIGN_PERFORMANCE_REPORT'
fields = ["AccountDescriptiveName","CampaignName","Date","HourOfDay","AdvertisingChannelType","AdNetworkType1","AdNetworkType2","Impressions", "Clicks", "Cost"]
response_Saatva = download_report_as_stream(client_Saatva, report_type, fields, Start_Date_overall, End_Data_overall)
Saatva_Click = pd.read_csv(response_Saatva)
Saatva_Click['Cost']=Saatva_Click['Cost'].apply(lambda x: x/1000000)
Saatva_Click['Channel_Network']=Saatva_Click['Advertising Channel']+"|"+Saatva_Click['Network']+"|"+Saatva_Click['Network (with search partners)']

Request Summary: {'clientCustomerId': '634-061-0672', 'includeZeroImpressions': False, 'server': 'adwords.google.com', 'skipColumnHeader': 'False', 'skipReportHeader': 'True', 'skipReportSummary': 'True', 'isError': True, 'errorMessage': '<?xml version="1.0" encoding="UTF-8" standalone="yes"?><reportDownloadError><ApiError><type>SelectorError.INVALID_PREDICATE_ENUM_VALUE</type><trigger>Search</trigger><fieldPath>selector.reportDefinition.selector</fieldPath></ApiError></reportDownloadError>'}


AdWordsReportBadRequestError: Type: SelectorError.INVALID_PREDICATE_ENUM_VALUE
Trigger: Search
Field Path: selector.reportDefinition.selector

In [40]:
Saatva_Click['Advertising Channel'].unique()

array(['Search', 'Shopping', 'Display', 'Video'], dtype=object)

In [41]:
client_Saatva_Branding = get_client(config_path='/home/jian/Projects/Saatva/To_Lexie_and_Connor/googleads_saatva_JL_Saatva_Branding.yaml')
report_type = 'CAMPAIGN_PERFORMANCE_REPORT'
fields = ["AccountDescriptiveName","CampaignName","Date","HourOfDay","AdvertisingChannelType","AdNetworkType1","AdNetworkType2","Impressions", "Clicks", "Cost"]
response_Saatva_Branding = download_report_as_stream(client_Saatva_Branding, report_type, fields, Start_Date_overall, End_Data_overall)
Saatva_Brand_Click = pd.read_csv(response_Saatva_Branding)
Saatva_Brand_Click['Cost']=Saatva_Brand_Click['Cost'].apply(lambda x: x/1000000)
Saatva_Brand_Click['Channel_Network']=Saatva_Brand_Click['Advertising Channel']+"|"+Saatva_Brand_Click['Network']+"|"+Saatva_Brand_Click['Network (with search partners)']


In [42]:
client_Saatva_Competitors = get_client(config_path='/home/jian/Projects/Saatva/To_Lexie_and_Connor/googleads_saatva_JL_Saatva_Competitors.yaml')
report_type = 'CAMPAIGN_PERFORMANCE_REPORT'
fields = ["AccountDescriptiveName","CampaignName","Date","HourOfDay","AdvertisingChannelType","AdNetworkType1","AdNetworkType2","Impressions", "Clicks", "Cost"]
response_Saatva_Competitor = download_report_as_stream(client_Saatva_Competitors, report_type, fields, Start_Date_overall, End_Data_overall)
Saatva_Competitor_Click = pd.read_csv(response_Saatva_Competitor)
Saatva_Competitor_Click['Cost']=Saatva_Competitor_Click['Cost'].apply(lambda x: x/1000000)
Saatva_Competitor_Click['Channel_Network']=Saatva_Competitor_Click['Advertising Channel']+"|"+Saatva_Competitor_Click['Network']+"|"+Saatva_Competitor_Click['Network (with search partners)']



In [43]:
client_TheSaatvaCompany = get_client(config_path='/home/jian/Projects/Saatva/To_Lexie_and_Connor/googleads_saatva_JL_TheSaatvaCompany.yaml')
report_type = 'CAMPAIGN_PERFORMANCE_REPORT'
fields = ["AccountDescriptiveName","CampaignName","Date","HourOfDay","AdvertisingChannelType","AdNetworkType1","AdNetworkType2","Impressions", "Clicks", "Cost"]
response_TheSaatvaCompany = download_report_as_stream(client_TheSaatvaCompany, report_type, fields, Start_Date_overall, End_Data_overall)
TheSaatvaCompany_Click = pd.read_csv(response_TheSaatvaCompany)
TheSaatvaCompany_Click['Cost']=TheSaatvaCompany_Click['Cost'].apply(lambda x: x/1000000)
TheSaatvaCompany_Click['Channel_Network']=TheSaatvaCompany_Click['Advertising Channel']+"|"+TheSaatvaCompany_Click['Network']+"|"+TheSaatvaCompany_Click['Network (with search partners)']



In [44]:
client_TheSaatvaCompanyBranding = get_client(config_path='/home/jian/Projects/Saatva/To_Lexie_and_Connor/googleads_saatva_JL_TheSaatvaCompanyBranding.yaml')
report_type = 'CAMPAIGN_PERFORMANCE_REPORT'
fields = ["AccountDescriptiveName","CampaignName","Date","HourOfDay","AdvertisingChannelType","AdNetworkType1","AdNetworkType2","Impressions", "Clicks", "Cost"]
response_TheSaatvaCompanyBranding = download_report_as_stream(client_TheSaatvaCompanyBranding, report_type, fields, Start_Date_overall, End_Data_overall)
TheSaatvaCompanyBranding_Click = pd.read_csv(response_TheSaatvaCompanyBranding)
TheSaatvaCompanyBranding_Click['Cost']=TheSaatvaCompanyBranding_Click['Cost'].apply(lambda x: x/1000000)
TheSaatvaCompanyBranding_Click['Channel_Network']=TheSaatvaCompanyBranding_Click['Advertising Channel']+"|"+TheSaatvaCompanyBranding_Click['Network']+"|"+TheSaatvaCompanyBranding_Click['Network (with search partners)']



In [45]:
Saatva_Competitor_Click=Saatva_Competitor_Click[Saatva_Competitor_Click['Campaign'].apply(lambda x: x[len(x)-4:len(x)+1])=="[SM]"]
Saatva_Competitor_Click.reset_index(inplace=True)
del Saatva_Competitor_Click['index']
# Keep only campaign labeled as "[SM]" in Saatva Competitor account
# Broad and Exact ADL Non-Brand Search Partners are not included

In [46]:
set(Saatva_Click['Channel_Network'].unique().tolist()+Saatva_Brand_Click['Channel_Network'].unique().tolist()+Saatva_Competitor_Click['Channel_Network'].unique().tolist()+\
    TheSaatvaCompany_Click['Channel_Network'].unique().tolist()+TheSaatvaCompanyBranding_Click['Channel_Network'].unique().tolist())

{'Display|Display Network|Display Network',
 'Search|Search Network|Google search',
 'Search|Search Network|Search partners',
 'Shopping|Search Network|Google search',
 'Shopping|Search Network|Search partners',
 'Video|Display Network|Display Network',
 'Video|YouTube Search|YouTube Search',
 'Video|YouTube Videos|YouTube Videos'}

In [17]:
df_all_adwords=Saatva_Click.append(Saatva_Brand_Click).append(Saatva_Competitor_Click)
df_all_adwords=df_all_adwords.append(TheSaatvaCompany_Click).append(TheSaatvaCompanyBranding_Click)

In [18]:
df_s=df_all_adwords.groupby(['Channel_Network'])[['Impressions','Clicks','Cost']].sum().reset_index()
df_s[df_s['Channel_Network'].apply(lambda x: "Video" in x)]

,Channel_Network,Impressions,Clicks,Cost
5,Video|Display Network|Display Network,6540,17,34.395750
6,Video|YouTube Search|YouTube Search,435680,30,1694.220000
7,Video|YouTube Videos|YouTube Videos,17447212,69224,220694.240888


In [19]:
df_all_adwords.head(2)

,Account,Campaign,Day,Hour of day,Advertising Channel,Network,Network (with search partners),Impressions,Clicks,Cost,Channel_Network
0,Saatva,[ADL Shopping] Saatva - {Neither} - (Desktop),2018-09-30,6,Shopping,Search Network,Search partners,65,1,4.55,Shopping|Search Network|Search partners
1,Saatva,[ADL] [Display] [US] SCM - Conservative [CPA],2018-09-14,21,Display,Display Network,Display Network,10532,28,93.40,Display|Display Network|Display Network


In [20]:
df_all_adwords[df_all_adwords['Channel_Network']=="Video|Display Network|Display Network"]['Cost'].sum()

34.395750000000007

In [21]:
df_all_adwords[df_all_adwords['Channel_Network']=="Video|YouTube Search|YouTube Search"]['Cost'].sum()

1694.2200000000003

In [22]:
df_all_adwords[df_all_adwords['Channel_Network']=="Video|YouTube Videos|YouTube Videos"]['Cost'].sum()

220694.24088799997

In [23]:
df_all_adwords[df_all_adwords['Channel_Network']=="Video|Display Network|Display Network"]['Day'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date()).min()

datetime.date(2018, 9, 11)

In [24]:
df_all_adwords[df_all_adwords['Channel_Network']=="Video|Display Network|Display Network"]['Day'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date()).max()

datetime.date(2018, 9, 19)

In [25]:
df_all_adwords[df_all_adwords['Channel_Network']=="Video|YouTube Search|YouTube Search"]['Day'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date()).min()

datetime.date(2018, 9, 11)

In [26]:
df_all_adwords[df_all_adwords['Channel_Network']=="Video|YouTube Search|YouTube Search"]['Day'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date()).max()

datetime.date(2018, 10, 11)

In [27]:
df_all_adwords[df_all_adwords['Channel_Network']=="Video|YouTube Videos|YouTube Videos"]['Day'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date()).min()

datetime.date(2018, 9, 11)

In [28]:
df_all_adwords[df_all_adwords['Channel_Network']=="Video|YouTube Videos|YouTube Videos"]['Day'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date()).max()

datetime.date(2018, 10, 11)

In [29]:
df_x=df_all_adwords[df_all_adwords['Channel_Network']=="Video|YouTube Videos|YouTube Videos"]
df_x['Day']=df_x['Day'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
df_x[df_x['Day']<datetime.date(2018,9,19)]['Cost'].sum()

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


9427.5978470000009

In [30]:
df_x[df_x['Day']<datetime.date(2018,9,19)]

,Account,Campaign,Day,Hour of day,Advertising Channel,Network,Network (with search partners),Impressions,Clicks,Cost,Channel_Network
12,Saatva,[ADL] [Video] [Bumper] [US] Remarketing (Display),2018-09-16,5,Video,YouTube Videos,YouTube Videos,102,0,0.652565,Video|YouTube Videos|YouTube Videos
28,Saatva,[ADL] [Video] [Bumper] [US] Remarketing (Display),2018-09-11,13,Video,YouTube Videos,YouTube Videos,710,1,5.091620,Video|YouTube Videos|YouTube Videos
30,Saatva,[ADL] [Video] [Bumper] [US] Remarketing (Display),2018-09-16,19,Video,YouTube Videos,YouTube Videos,352,1,2.152164,Video|YouTube Videos|YouTube Videos
32,Saatva,[ADL] [Video] [Bumper] [US] Remarketing (Display),2018-09-17,7,Video,YouTube Videos,YouTube Videos,156,0,1.038030,Video|YouTube Videos|YouTube Videos
66,Saatva,[ADL] [Video] [Bumper] [US] Remarketing (Display),2018-09-11,1,Video,YouTube Videos,YouTube Videos,82,0,0.464780,Video|YouTube Videos|YouTube Videos
77,Saatva,[ADL] [Video] [Bumper] [US] Remarketing (Display),2018-09-14,21,Video,YouTube Videos,YouTube Videos,442,0,2.842245,Video|YouTube Videos|YouTube Videos
82,Saatva,[ADL] [Video] [Bumper] [US] Remarketing (Display),2018-09-12,9,Video,YouTube Videos,YouTube Videos,174,0,1.119440,Video|YouTube Videos|YouTube Videos
83,Saatva,[ADL] [Video] [Bumper] [US] Remarketing (Display),2018-09-18,2,Video,YouTube Videos,YouTube Videos,44,0,0.276520,Video|YouTube Videos|YouTube Videos
90,Saatva,[ADL] [Video] [Bumper] [US] Remarketing (Display),2018-09-12,8,Video,YouTube Videos,YouTube Videos,153,0,0.972000,Video|YouTube Videos|YouTube Videos
104,Saatva,[ADL] [Video] [Bumper] [US] Remarketing (Display),2018-09-12,7,Video,YouTube Videos,YouTube Videos,113,0,0.706660,Video|YouTube Videos|YouTube Videos


In [31]:
'''
Saatva_Click=Saatva_Click[~((Saatva_Click['Campaign'].str.contains("Loom")) | (Saatva_Click['Campaign'].str.contains("Leaf")) | (Saatva_Click['Campaign'].str.contains("Zenhaven")))]
TheSaatvaCompany_Click=TheSaatvaCompany_Click[~((TheSaatvaCompany_Click['Campaign'].str.contains("Loom")) | (TheSaatvaCompany_Click['Campaign'].str.contains("Leaf")) | (TheSaatvaCompany_Click['Campaign'].str.contains("Zenhaven")))]
TheSaatvaCompanyBranding_Click=TheSaatvaCompanyBranding_Click[~((TheSaatvaCompanyBranding_Click['Campaign'].str.contains("Loom")) | (TheSaatvaCompanyBranding_Click['Campaign'].str.contains("Leaf")) | (TheSaatvaCompanyBranding_Click['Campaign'].str.contains("Zenhaven")))]
'''
# No need to exclude the compaigns because of the small pctg

Saatva_Click['Campaign_Type']=Saatva_Click['Advertising Channel'].apply(lambda x: x.replace("Search","SEM").replace("Shopping","PLA"))
Saatva_Brand_Click['Campaign_Type']=Saatva_Brand_Click['Advertising Channel'].apply(lambda x: x.replace("Search","SEM_Brand").replace("Shopping","PLA"))
Saatva_Competitor_Click['Campaign_Type']="SEM_Non_Brand" # Because it's all labeled as Non-Brand in Campaign and in Search Only Campaign Type
TheSaatvaCompany_Click['Campaign_Type']=TheSaatvaCompany_Click['Advertising Channel'].apply(lambda x: x.replace("Search","SEM").replace("Shopping","PLA"))
TheSaatvaCompanyBranding_Click['Campaign_Type']=TheSaatvaCompanyBranding_Click['Advertising Channel'].apply(lambda x: x.replace("Search","SEM").replace("Shopping","PLA"))


In [32]:
# Split Saatva
# 'Search Network with Display Select' Campaign Type all 0 Impr and Clicks
Saatva_Click_Search=Saatva_Click[Saatva_Click['Advertising Channel']=='Search']
Saatva_Click_Others=Saatva_Click[Saatva_Click['Advertising Channel']!='Search']

Saatva_Click_Search['Campaign_Type']=np.where((Saatva_Click_Search['Campaign'].str.contains("Branded")) | (Saatva_Click_Search['Campaign'].str.contains(" Brand ")) | (Saatva_Click_Search['Campaign'].str.contains("Co-Brand")) | (Saatva_Click_Search['Campaign'].str.contains("\[Brand\]")),"SEM_Brand",
                                                 np.where((Saatva_Click_Search['Campaign'].str.contains("Non-Brand")) | (Saatva_Click_Search['Campaign'].str.contains("Dynamic Search Ads")),"SEM_Non_Brand","NA"))
Saatva_Click_Search_1=Saatva_Click_Search[Saatva_Click_Search['Campaign_Type']!="NA"]
Saatva_Click_Search_2=Saatva_Click_Search[Saatva_Click_Search['Campaign_Type']=="NA"]
len(Saatva_Click_Search_2) # Make sure it's 0

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


0

In [33]:
# Split TheSaatvaCompany

TheSaatvaCompany_Click_Search=TheSaatvaCompany_Click[TheSaatvaCompany_Click['Advertising Channel']=='Search']
TheSaatvaCompany_Click_Others=TheSaatvaCompany_Click[TheSaatvaCompany_Click['Advertising Channel']!='Search']

TheSaatvaCompany_Click_Search['Campaign_Type']=np.where((TheSaatvaCompany_Click_Search['Campaign'].str.contains("Branded")) | (TheSaatvaCompany_Click_Search['Campaign'].str.contains(" Brand ")) | (TheSaatvaCompany_Click_Search['Campaign'].str.contains("Co-Brand")) | (TheSaatvaCompany_Click_Search['Campaign'].str.contains("\[Brand\]")),"SEM_Brand",
                                                 np.where((TheSaatvaCompany_Click_Search['Campaign'].str.contains("Non-Brand")) | (TheSaatvaCompany_Click_Search['Campaign'].str.contains("Dynamic Search Ads")),"SEM_Non_Brand","NA"))
TheSaatvaCompany_Click_Search_1=TheSaatvaCompany_Click_Search[TheSaatvaCompany_Click_Search['Campaign_Type']!="NA"]
TheSaatvaCompany_Click_Search_2=TheSaatvaCompany_Click_Search[TheSaatvaCompany_Click_Search['Campaign_Type']=="NA"]
len(TheSaatvaCompany_Click_Search_2) # Make sure it's 0

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


0

In [34]:
# All campaing is able to differentiate from the name about brand and non-brand, so no need to go to keywords level
'''
Saatva_keywords=pd.read_csv("/home/jian/Projects/Saatva/To_Lexie_and_Connor/AdWords/saatva_keywords.csv",dtype=str,skiprows=2)
Saatva_Brand_keywords=pd.read_csv("/home/jian/Projects/Saatva/To_Lexie_and_Connor/AdWords/saatva branding_keywords.csv",dtype=str)
Saatva_Competitor_keywords=pd.read_csv("/home/jian/Projects/Saatva/To_Lexie_and_Connor/AdWords/saatva competitors_keywords.csv",dtype=str,skiprows=2)

Saatva_keywords=Saatva_keywords[~((Saatva_keywords['Campaign'].str.contains("Loom")) | (Saatva_keywords['Campaign'].str.contains("Leaf")) | (Saatva_keywords['Campaign'].str.contains("Zenhaven")))]
Saatva_keywords['Campaign_Type']=np.where(Saatva_keywords['Search keyword'].str.contains("aatva"),"SEM_Brand","SEM_Non_Brand")
Saatva_keywords['Impressions']=Saatva_keywords['Impressions'].apply(lambda x: int(x.replace(",","")))
Saatva_keywords['Impressions']=Saatva_keywords['Impressions'].astype(int)
Saatva_keywords['Clicks']=Saatva_keywords['Clicks'].apply(lambda x: int(x.replace(",","")))
Saatva_keywords['Clicks']=Saatva_keywords['Clicks'].astype(int)
Saatva_keywords=Saatva_keywords.groupby(['Campaign','Campaign_Type'])['Impressions','Clicks'].sum()
Saatva_keywords.reset_index(inplace=True)
Saatva_keywords=Saatva_keywords.sort_values(['Campaign','Impressions'])
Saatva_keywords=Saatva_keywords.drop_duplicates('Campaign')
Saatva_Campaign_Type=Saatva_keywords[['Campaign','Campaign_Type']]

del Saatva_Click_Search_2['Campaign_Type']
Saatva_Click_Search_2=pd.merge(Saatva_Click_Search_2,Saatva_Campaign_Type,on="Campaign",how="left")

Saatva_Search_Click=Saatva_Click_Search_1.append(Saatva_Click_Search_2)
Saatva_Search_Click['Campaign_Type']=Saatva_Search_Click['Campaign_Type'].fillna("All_0_Values_and_No_Kws")
'''

'\nSaatva_keywords=pd.read_csv("/home/jian/Projects/Saatva/To_Lexie_and_Connor/AdWords/saatva_keywords.csv",dtype=str,skiprows=2)\nSaatva_Brand_keywords=pd.read_csv("/home/jian/Projects/Saatva/To_Lexie_and_Connor/AdWords/saatva branding_keywords.csv",dtype=str)\nSaatva_Competitor_keywords=pd.read_csv("/home/jian/Projects/Saatva/To_Lexie_and_Connor/AdWords/saatva competitors_keywords.csv",dtype=str,skiprows=2)\n\nSaatva_keywords=Saatva_keywords[~((Saatva_keywords[\'Campaign\'].str.contains("Loom")) | (Saatva_keywords[\'Campaign\'].str.contains("Leaf")) | (Saatva_keywords[\'Campaign\'].str.contains("Zenhaven")))]\nSaatva_keywords[\'Campaign_Type\']=np.where(Saatva_keywords[\'Search keyword\'].str.contains("aatva"),"SEM_Brand","SEM_Non_Brand")\nSaatva_keywords[\'Impressions\']=Saatva_keywords[\'Impressions\'].apply(lambda x: int(x.replace(",","")))\nSaatva_keywords[\'Impressions\']=Saatva_keywords[\'Impressions\'].astype(int)\nSaatva_keywords[\'Clicks\']=Saatva_keywords[\'Clicks\'].apply(

In [35]:
Saatva_Click_Search_2.head(2)

,Account,Campaign,Day,Hour of day,Advertising Channel,Network,Network (with search partners),Impressions,Clicks,Cost,Channel_Network,Campaign_Type


In [36]:
Saatva_Click=Saatva_Click_Search.append(Saatva_Click_Others)
TheSaatvaCompany_Click=TheSaatvaCompany_Click_Search.append(TheSaatvaCompany_Click_Others)


In [37]:
writer=pd.ExcelWriter(writer_folder+"/Saatva_AdWords_"+Start_Date_overall+"_to_"+End_Data_overall+week+"_cost.xlsx",engine='xlsxwriter')

In [39]:
Saatva_Agg_Category.head(2)

,Campaign,Account,Advertising Channel,Campaign_Type
0,[ADL] [Co-Brand] [US] Zenhaven (Exact),Saatva,Search,SEM_Brand
1,[ADL] [Non-Brand] [US] Landing Page Tests (Broad),Saatva,Search,SEM_Non_Brand


In [38]:
Saatva_Agg=Saatva_Click.append(Saatva_Brand_Click).append(Saatva_Competitor_Click).append(TheSaatvaCompany_Click).append(TheSaatvaCompanyBranding_Click)
Saatva_Agg['Day']=Saatva_Agg['Day'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
Saatva_Agg.reset_index(inplace=True)
del Saatva_Agg['index']
Saatva_Agg['Hour of day']=Saatva_Agg['Hour of day'].astype(int)
Saatva_Agg_Category=Saatva_Agg[['Campaign','Account','Advertising Channel','Campaign_Type']].drop_duplicates()
Saatva_Agg_Category.to_excel(writer,"Campaign Category",index=False)

In [55]:
Saatva_Agg=Saatva_Agg.groupby(["Campaign_Type","Day","Hour of day"])['Impressions','Clicks', 'Cost'].apply(sum)
Saatva_Agg.reset_index(inplace=True)
Saatva_Agg['CTR']=Saatva_Agg['Clicks']/Saatva_Agg['Impressions']
Saatva_Agg.to_excel(writer,"reshaped by hour day",index=False)

Saatva_Agg_No_NA=Saatva_Agg[(Saatva_Agg['Campaign_Type']!="NA") & (Saatva_Agg['Campaign_Type']!="Search Network with Display Select")]

Saatva_Agg_No_NA.to_excel(writer,"clean data by hour day",index=False)

In [56]:
writer.save()

In [57]:
AdWords_output=Saatva_Agg.copy()
AdWords_output['Date']=AdWords_output['Day'].apply(lambda x: str(x)[0:4]+str(x)[5:7]+str(x)[8:10])
AdWords_output=AdWords_output[['Campaign_Type','Date','Hour of day','Impressions','Clicks','Cost']]
AdWords_output=AdWords_output.rename(columns={"Hour of day":"Hour","Impressions":"IMP"})
AdWords_output=AdWords_output.sort_values(["Campaign_Type","Date","Hour"],ascending=[True,True,True])
AdWords_output.to_csv(writer_folder+"/Saatva_AdWords_"+Start_Date_overall+"_to_"+End_Data_overall+week+"_cost.csv",index=False)